In [203]:
import pandas as pd
import pandas_flavor as pf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import janitor
# from janitor import remove_columns, rename_column, remove_empty, filter_on, groupby_agg, clean_names

In [204]:
birth_rate_df = pd.read_csv('../data/birth_rate_cl.csv')
birth_rate_df.head(10)

,country,year,birth_rate_per_thousand
0,Afghanistan,1979,48.1
1,Afghanistan,2000,41.2
2,Afghanistan,2009,35.6
3,Albania,1970,32.5
4,Albania,1985,26.2
5,Albania,1995,23.6
6,Albania,2005,12.6
7,Albania,2011,10.1
8,Algeria,1970,43.9
9,Algeria,1985,38.3


In [205]:
fertility_rate_df = pd.read_csv('../data/fertility_rates_cl.csv')
fertility_rate_df.head(10)

,country,total_fertility,mean_age_at_childbearing_,year
0,Afghanistan,8.21,30.0,1970
1,Afghanistan,7.52,30.1,2005
2,Afghanistan,5.11,29.1,2020
3,Albania,5.16,29.7,1970
4,Albania,3.33,28.8,1985
5,Albania,2.86,27.6,1995
6,Albania,1.61,27.7,2005
7,Albania,1.33,28.2,2020
8,Algeria,7.86,30.4,1970
9,Algeria,5.35,31.8,1985


In [206]:
gdp_df = (
    pd.read_csv('../data/gdp_cl.csv')
    .rename(columns={'Country': 'country', 'Year': 'year'})
)
gdp_df.head(10)

,country,year,GDP_Per_Capita
0,Afghanistan,2021,372.548875
1,Afghanistan,2020,516.866543
2,Afghanistan,2019,500.522664
3,Afghanistan,2018,502.056771
4,Afghanistan,2017,530.149831
5,Afghanistan,2016,520.252064
6,Afghanistan,2015,554.403216
7,Afghanistan,2014,597.584579
8,Afghanistan,2013,622.100518
9,Afghanistan,2012,628.116539


In [207]:
gdp_df = gdp_df.groupby(gdp_df['year'] // 10 * 10).agg({'GDP_Per_Capita': 'mean'}).reset_index()
gdp_df['year'] = gdp_df['year'] + 9  
gdp_df.head(10)

,year,GDP_Per_Capita
0,1979,2254.240367
1,1989,4723.050400
2,1999,7458.257812
3,2009,12055.854436
4,2019,16920.511144
5,2029,17427.621999


In [208]:
merged_df = birth_rate_df.merge(fertility_rate_df, on='year', how='inner')

# merged_df.fillna(method='ffill', inplace=True) 

merged_df

,country_x,year,birth_rate_per_thousand,country_y,total_fertility,mean_age_at_childbearing_
0,Albania,1970,32.5,Afghanistan,8.21,30.0
1,Albania,1970,32.5,Albania,5.16,29.7
2,Albania,1970,32.5,Algeria,7.86,30.4
3,Albania,1970,32.5,American Samoa,5.83,30.7
4,Albania,1970,32.5,Antigua and Barbuda,2.68,27.1
...,...,...,...,...,...,...
111383,Wallis and Futuna Islands,2005,15.7,Viet Nam,2.08,27.3
111384,Wallis and Futuna Islands,2005,15.7,Wallis and Futuna Islands,2.12,29.1
111385,Wallis and Futuna Islands,2005,15.7,Yemen,6.16,30.8
111386,Wallis and Futuna Islands,2005,15.7,Zambia,5.87,28.9
